### Import Libraries

In [36]:
// Import Libraries

import scala.io.Source
import scala.util.{Try, Using}
import java.time.LocalDate

import scala.io.Source

import scala.util.{Try, Using}

import java.time.LocalDate


### Import Files

In [37]:
// Retrieving the File Path (using lazy val)

val configPath = "config.txt" // Retrieve file path from config.txt
lazy val filePath = Try(Source.fromFile(configPath).getLines().next()).getOrElse("songs_2000_2020_50k.csv") // Get file path, suppress output


configPath: String = "config.txt"
filePath: String = <lazy>

### Data Cleaning - Part 1

In [38]:
// Reading and Parsing the Data

val dataRows = Using(Source.fromFile(filePath)) { source =>
  source.getLines().toList
}.getOrElse(List())

val header = dataRows.head.split(",").map(_.trim) // First row is taken as headers
val data = dataRows.tail.map(_.split(",").map(_.trim).toList) // Remaining rows are treated as data


dataRows: List[String] = List(
  "Title,Artist,Album,Genre,Release Date,Duration,Popularity",
  "Include name this.,Patrick Anderson,Care.,R&B,2008-01-09,262,71",
  "Manage west energy.,Eric Miller,Raise get.,Jazz,2011-08-20,187,37",
  "Evening court painting.,Richard Curry,Sport.,Electronic,2010-05-30,212,58",
  "Section turn hour.,James Smith,Full.,Hip-Hop,2014-10-12,272,59",
  "Five agreement teach.,Amy Rodriguez,Eat.,Blues,2005-06-09,131,34",
  "Turn child.,Jessica Martin,Cold according.,R&B,2006-09-16,207,58",
  "Old.,Cheyenne Powell,Oil.,Country,2010-04-23,163,72",
  "Clear fly over.,Aaron Coleman,Strategy development.,Classical,2010-02-06,183,73",
  "Agency employee present.,Brandon Henderson,Might live.,Country,2020-02-18,243,69",
  "Face become we.,Raymond White,Probably camera.,Blues,2011-11-07,177,55",
  "Couple bank.,Paul Stephens,And.,Reggae,2016-03-31,245,76",
  "Wife subject.,Julie Martin,Of.,Rock,2003-01-27,191,97",
  "Business research.,Michael Glass,Speak.,Blues,2019-

In [39]:
// Cleaning and Transforming Data

val cleanedData = data.map { row =>
  val rawData = header.zip(row).toMap
  
  Map(
    "Title" -> rawData.getOrElse("Title", "Unknown Title"),
    "Artist" -> rawData.getOrElse("Artist", "Unknown Artist"),
    "Album" -> rawData.getOrElse("Album", "Unknown Album"),
    "Genre" -> rawData.getOrElse("Genre", "Unknown Genre"),
    "Duration" -> Try(rawData.getOrElse("Duration", "0").toInt).getOrElse(0).toString,
    "Popularity" -> Try(rawData.getOrElse("Popularity", "0").toInt).getOrElse(0).toString,
    "Release Date" -> Try(LocalDate.parse(rawData.getOrElse("Release Date", "2000-01-01"))).getOrElse(LocalDate.of(2000, 1, 1)).toString
  )
}

cleanedData: List[Map[String, String]] = List(
  HashMap(
    "Title" -> "Include name this.",
    "Duration" -> "262",
    "Popularity" -> "71",
    "Album" -> "Care.",
    "Artist" -> "Patrick Anderson",
    "Release Date" -> "2008-01-09",
    "Genre" -> "R&B"
  ),
  HashMap(
    "Title" -> "Manage west energy.",
    "Duration" -> "187",
    "Popularity" -> "37",
    "Album" -> "Raise get.",
    "Artist" -> "Eric Miller",
    "Release Date" -> "2011-08-20",
    "Genre" -> "Jazz"
  ),
  HashMap(
    "Title" -> "Evening court painting.",
    "Duration" -> "212",
    "Popularity" -> "58",
    "Album" -> "Sport.",
    "Artist" -> "Richard Curry",
    "Release Date" -> "2010-05-30",
    "Genre" -> "Electronic"
  ),
  HashMap(
    "Title" -> "Section turn hour.",
    "Duration" -> "272",
    "Popularity" -> "59",
    "Album" -> "Full.",
    "Artist" -> "James Smith",
    "Release Date" -> "2014-10-12",
    "Genre" -> "Hip-Hop"
  ),
  HashMap(
    "Title" -> "Five agreement teach.",
...

In [40]:
// Printing Results

// Print headers
println(header.mkString(" | "))

// Print a separating line
println("-" * 50)

// Print the first 5 rows
cleanedData.take(5).foreach { row =>
  println(header.map(h => row.getOrElse(h, "")).mkString(" | "))
}

Title | Artist | Album | Genre | Release Date | Duration | Popularity
--------------------------------------------------
Include name this. | Patrick Anderson | Care. | R&B | 2008-01-09 | 262 | 71
Manage west energy. | Eric Miller | Raise get. | Jazz | 2011-08-20 | 187 | 37
Evening court painting. | Richard Curry | Sport. | Electronic | 2010-05-30 | 212 | 58
Section turn hour. | James Smith | Full. | Hip-Hop | 2014-10-12 | 272 | 59
Five agreement teach. | Amy Rodriguez | Eat. | Blues | 2005-06-09 | 131 | 34


### Data Cleaning - Part 2

In [41]:
// Transforming the cleaned data
val transformedData = cleanedData.map { row =>
  // Define popularity categories based on the "Popularity" value
  val popularityCategory = row("Popularity").toInt match {
    case pop if pop >= 70 => "High"
    case pop if pop >= 40 => "Medium"
    case _ => "Low"
  }

  // Extract year from "Release Date"
  val releaseYear = LocalDate.parse(row("Release Date")).getYear.toString

  // Normalize title and artist (capitalize each word)
  val titleNormalized = row("Title").split(" ").map(_.capitalize).mkString(" ")
  val artistNormalized = row("Artist").split(" ").map(_.capitalize).mkString(" ")

  // Add the new transformations to the row
  row ++ Map(
    "Popularity Category" -> popularityCategory,
    "Release Year" -> releaseYear,
    "Title Normalized" -> titleNormalized,
    "Artist Normalized" -> artistNormalized
  )
}

// Print transformed headers and the first 5 rows of transformed data
val transformedHeader = header ++ Seq("Popularity Category", "Release Year", "Title Normalized", "Artist Normalized")
println(transformedHeader.mkString(" | "))
println("-" * 50)
transformedData.take(5).foreach { row =>
  println(transformedHeader.map(h => row.getOrElse(h, "")).mkString(" | "))
}


Title | Artist | Album | Genre | Release Date | Duration | Popularity | Popularity Category | Release Year | Title Normalized | Artist Normalized
--------------------------------------------------
Include name this. | Patrick Anderson | Care. | R&B | 2008-01-09 | 262 | 71 | High | 2008 | Include Name This. | Patrick Anderson
Manage west energy. | Eric Miller | Raise get. | Jazz | 2011-08-20 | 187 | 37 | Low | 2011 | Manage West Energy. | Eric Miller
Evening court painting. | Richard Curry | Sport. | Electronic | 2010-05-30 | 212 | 58 | Medium | 2010 | Evening Court Painting. | Richard Curry
Section turn hour. | James Smith | Full. | Hip-Hop | 2014-10-12 | 272 | 59 | Medium | 2014 | Section Turn Hour. | James Smith
Five agreement teach. | Amy Rodriguez | Eat. | Blues | 2005-06-09 | 131 | 34 | Low | 2005 | Five Agreement Teach. | Amy Rodriguez


transformedData: List[Map[String, String]] = List(
  HashMap(
    "Title" -> "Include name this.",
    "Duration" -> "262",
    "Title Normalized" -> "Include Name This.",
    "Popularity" -> "71",
    "Album" -> "Care.",
    "Artist" -> "Patrick Anderson",
    "Release Year" -> "2008",
    "Release Date" -> "2008-01-09",
    "Genre" -> "R&B",
    "Artist Normalized" -> "Patrick Anderson",
    "Popularity Category" -> "High"
  ),
  HashMap(
    "Title" -> "Manage west energy.",
    "Duration" -> "187",
    "Title Normalized" -> "Manage West Energy.",
    "Popularity" -> "37",
    "Album" -> "Raise get.",
    "Artist" -> "Eric Miller",
    "Release Year" -> "2011",
    "Release Date" -> "2011-08-20",
    "Genre" -> "Jazz",
    "Artist Normalized" -> "Eric Miller",
    "Popularity Category" -> "Low"
  ),
  HashMap(
    "Title" -> "Evening court painting.",
    "Duration" -> "212",
    "Title Normalized" -> "Evening Court Painting.",
    "Popularity" -> "58",
    "Album" -> "Sport.",
    

### Data Cleaning - Part 3

In [42]:
// Group by Genre and Release Year to calculate average Popularity and song count
val genreYearStats = transformedData
  .groupBy(row => (row("Genre"), row("Release Year")))
  .view.mapValues { rows =>
    val popularities = rows.map(_("Popularity").toInt)
    
    // Calculate average popularity and song count for each group
    val avgPopularity = popularities.sum / popularities.size
    val songCount = rows.size
    
    Map(
      "Average Popularity by Genre-Year" -> avgPopularity.toString,
      "Song Count by Genre-Year" -> songCount.toString
    )
  }.toMap

// Convert grouped stats to a list of maps for merging
val genreYearStatsData = genreYearStats.toList.map {
  case ((genre, year), stats) =>
    Map(
      "Genre" -> genre,
      "Release Year" -> year,
      "Average Popularity by Genre-Year" -> stats("Average Popularity by Genre-Year"),
      "Song Count by Genre-Year" -> stats("Song Count by Genre-Year")
    )
}

// Print the grouped stats (first 5 rows for preview)
val statsHeader = Seq("Genre", "Release Year", "Average Popularity by Genre-Year", "Song Count by Genre-Year")
println(statsHeader.mkString(" | "))
println("-" * 60)
genreYearStatsData.take(5).foreach { row =>
  println(statsHeader.map(h => row.getOrElse(h, "")).mkString(" | "))
}

Genre | Release Year | Average Popularity by Genre-Year | Song Count by Genre-Year
------------------------------------------------------------
Classical | 2008 | 48 | 217
Reggae | 2005 | 50 | 234
Hip-Hop | 2017 | 50 | 223
Hip-Hop | 2003 | 51 | 233
Jazz | 2010 | 49 | 258


genreYearStats: Map[(String, String), Map[String, String]] = HashMap(
  ("Classical", "2008") -> Map(
    "Average Popularity by Genre-Year" -> "48",
    "Song Count by Genre-Year" -> "217"
  ),
  ("Reggae", "2005") -> Map(
    "Average Popularity by Genre-Year" -> "50",
    "Song Count by Genre-Year" -> "234"
  ),
  ("Hip-Hop", "2017") -> Map(
    "Average Popularity by Genre-Year" -> "50",
    "Song Count by Genre-Year" -> "223"
  ),
  ("Hip-Hop", "2003") -> Map(
    "Average Popularity by Genre-Year" -> "51",
    "Song Count by Genre-Year" -> "233"
  ),
  ("Jazz", "2010") -> Map(
    "Average Popularity by Genre-Year" -> "49",
    "Song Count by Genre-Year" -> "258"
  ),
  ("Country", "2014") -> Map(
    "Average Popularity by Genre-Year" -> "48",
    "Song Count by Genre-Year" -> "265"
  ),
  ("Jazz", "2012") -> Map(
    "Average Popularity by Genre-Year" -> "50",
    "Song Count by Genre-Year" -> "246"
  ),
  ("Rock", "2006") -> Map(
    "Average Popularity by Genre-Year" -> "49",

### Data Cleaning - Part 4

In [43]:
// Convert genreYearStatsData to a Map for easy lookup during merge
val genreYearStatsMap = genreYearStatsData.map { row =>
  (row("Genre"), row("Release Year")) -> row
}.toMap

// Merge transformedData with genreYearStatsMap based on Genre and Release Year
val mergedData = transformedData.map { row =>
  val key = (row("Genre"), row("Release Year"))
  val statsRow = genreYearStatsMap.getOrElse(key, Map(
    "Average Popularity by Genre-Year" -> "N/A",
    "Song Count by Genre-Year" -> "0"
  ))
  
  // Merge the original row with genre-year stats
  row ++ statsRow
}

// Print merged data (first 5 rows for preview)
val mergedHeader = transformedHeader ++ Seq("Average Popularity by Genre-Year", "Song Count by Genre-Year")
println(mergedHeader.mkString(" | "))
println("-" * 80)
mergedData.take(5).foreach { row =>
  println(mergedHeader.map(h => row.getOrElse(h, "")).mkString(" | "))
}

Title | Artist | Album | Genre | Release Date | Duration | Popularity | Popularity Category | Release Year | Title Normalized | Artist Normalized | Average Popularity by Genre-Year | Song Count by Genre-Year
--------------------------------------------------------------------------------
Include name this. | Patrick Anderson | Care. | R&B | 2008-01-09 | 262 | 71 | High | 2008 | Include Name This. | Patrick Anderson | 52 | 221
Manage west energy. | Eric Miller | Raise get. | Jazz | 2011-08-20 | 187 | 37 | Low | 2011 | Manage West Energy. | Eric Miller | 50 | 220
Evening court painting. | Richard Curry | Sport. | Electronic | 2010-05-30 | 212 | 58 | Medium | 2010 | Evening Court Painting. | Richard Curry | 52 | 219
Section turn hour. | James Smith | Full. | Hip-Hop | 2014-10-12 | 272 | 59 | Medium | 2014 | Section Turn Hour. | James Smith | 47 | 220
Five agreement teach. | Amy Rodriguez | Eat. | Blues | 2005-06-09 | 131 | 34 | Low | 2005 | Five Agreement Teach. | Amy Rodriguez | 48 | 265

genreYearStatsMap: Map[(String, String), Map[String, String]] = HashMap(
  ("Classical", "2008") -> Map(
    "Genre" -> "Classical",
    "Release Year" -> "2008",
    "Average Popularity by Genre-Year" -> "48",
    "Song Count by Genre-Year" -> "217"
  ),
  ("Reggae", "2005") -> Map(
    "Genre" -> "Reggae",
    "Release Year" -> "2005",
    "Average Popularity by Genre-Year" -> "50",
    "Song Count by Genre-Year" -> "234"
  ),
  ("Hip-Hop", "2017") -> Map(
    "Genre" -> "Hip-Hop",
    "Release Year" -> "2017",
    "Average Popularity by Genre-Year" -> "50",
    "Song Count by Genre-Year" -> "223"
  ),
  ("Hip-Hop", "2003") -> Map(
    "Genre" -> "Hip-Hop",
    "Release Year" -> "2003",
    "Average Popularity by Genre-Year" -> "51",
    "Song Count by Genre-Year" -> "233"
  ),
  ("Jazz", "2010") -> Map(
    "Genre" -> "Jazz",
    "Release Year" -> "2010",
    "Average Popularity by Genre-Year" -> "49",
    "Song Count by Genre-Year" -> "258"
  ),
  ("Country", "2014") -> Map(
    "G

### Data Cleaning - Part 5

In [44]:
// Step 1: Check and fill missing values if any
val finalData = mergedData.map { row =>
  row ++ Map(
    "Title" -> row.getOrElse("Title", "Unknown Title"),
    "Artist" -> row.getOrElse("Artist", "Unknown Artist"),
    "Album" -> row.getOrElse("Album", "Unknown Album"),
    "Genre" -> row.getOrElse("Genre", "Unknown Genre"),
    "Duration" -> row.getOrElse("Duration", "0"),
    "Popularity" -> row.getOrElse("Popularity", "0"),
    "Release Date" -> row.getOrElse("Release Date", "2000-01-01"),
    "Release Year" -> row.getOrElse("Release Year", "2000"),
    "Average Popularity by Genre-Year" -> row.getOrElse("Average Popularity by Genre-Year", "N/A"),
    "Song Count by Genre-Year" -> row.getOrElse("Song Count by Genre-Year", "0")
  )
}

// Step 2: Add Hit Song Flag based on Popularity
val finalDataWithHitFlag = finalData.map { row =>
  val isHitSong = row("Popularity").toInt >= 80 // Define threshold for "hit song"
  row + ("Is Hit Song" -> (if (isHitSong) "Yes" else "No"))
}

// Print final data headers and first 5 rows for preview
val finalHeader = mergedHeader ++ Seq("Is Hit Song")
println(finalHeader.mkString(" | "))
println("-" * 100)
finalDataWithHitFlag.take(5).foreach { row =>
  println(finalHeader.map(h => row.getOrElse(h, "")).mkString(" | "))
}


Title | Artist | Album | Genre | Release Date | Duration | Popularity | Popularity Category | Release Year | Title Normalized | Artist Normalized | Average Popularity by Genre-Year | Song Count by Genre-Year | Is Hit Song
----------------------------------------------------------------------------------------------------
Include name this. | Patrick Anderson | Care. | R&B | 2008-01-09 | 262 | 71 | High | 2008 | Include Name This. | Patrick Anderson | 52 | 221 | No
Manage west energy. | Eric Miller | Raise get. | Jazz | 2011-08-20 | 187 | 37 | Low | 2011 | Manage West Energy. | Eric Miller | 50 | 220 | No
Evening court painting. | Richard Curry | Sport. | Electronic | 2010-05-30 | 212 | 58 | Medium | 2010 | Evening Court Painting. | Richard Curry | 52 | 219 | No
Section turn hour. | James Smith | Full. | Hip-Hop | 2014-10-12 | 272 | 59 | Medium | 2014 | Section Turn Hour. | James Smith | 47 | 220 | No
Five agreement teach. | Amy Rodriguez | Eat. | Blues | 2005-06-09 | 131 | 34 | Low | 2

finalData: List[Map[String, String]] = List(
  HashMap(
    "Title" -> "Include name this.",
    "Duration" -> "262",
    "Title Normalized" -> "Include Name This.",
    "Album" -> "Care.",
    "Song Count by Genre-Year" -> "221",
    "Artist" -> "Patrick Anderson",
    "Release Year" -> "2008",
    "Release Date" -> "2008-01-09",
    "Genre" -> "R&B",
    "Artist Normalized" -> "Patrick Anderson",
    "Popularity Category" -> "High",
    "Average Popularity by Genre-Year" -> "52",
    "Popularity" -> "71"
  ),
  HashMap(
    "Title" -> "Manage west energy.",
    "Duration" -> "187",
    "Title Normalized" -> "Manage West Energy.",
    "Album" -> "Raise get.",
    "Song Count by Genre-Year" -> "220",
    "Artist" -> "Eric Miller",
    "Release Year" -> "2011",
    "Release Date" -> "2011-08-20",
    "Genre" -> "Jazz",
    "Artist Normalized" -> "Eric Miller",
    "Popularity Category" -> "Low",
    "Average Popularity by Genre-Year" -> "50",
    "Popularity" -> "37"
  ),
  HashMap(
   